<a href="https://colab.research.google.com/github/VilchisKarla/SIMULACION-I/blob/main/chi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **χ cuadrada**
*Vichis  García Karla Ivonne *


---

Prueba estadística  para números aleatorios
1. Agrupar los números aleatorios en k clases disjuntas de igual amplitud.

2. hallar la frecuancia de cada clase
3. calcular el estadístico

     χ ^2 = Σ [(fo-fe)^2/fe]

     para n suficientemente grande, este estadístico sigue la  distribución χ ^2 con (k-1)  grados de libertad
4. Determinar el valor  crícrítico χ ^2_(α,k-1) de las tablas para  un nivel de confianza (1-α).

     Si **χ ^2 >χ ^2_(α,k-1)** se rechaza H0

In [ ]:
import scipy.stats
import tabulate
def generar_clases(num_clases):
    minimo, maximo = 0, 1  # Rango de los datos
    ancho_clase = (maximo - minimo) / num_clases  # Tamaño de cada clase

    clases = []
    limites = [minimo + i * ancho_clase for i in range(num_clases)]

    # Ajusta los lmites derechos de las clases
    for i in range(num_clases):
        limite_inferior = limites[i]
        limite_superior = limites[i + 1] - 0.0001 if i < num_clases - 1 else maximo
        clases.append((limite_inferior, limite_superior))

    return clases

# Parámetros
N = 25  # Cantidad de elementos
num_clases = 5  # Número de clases
muestra = [0.951, 0.932, 0.926, 0.787, 0.772, 0.631, 0.594, 0.591, 0.551, 0.545, 0.533, 0.478, 0.437, 0.411, 0.401, 0.368, 0.362, 0.298, 0.199, 0.178, 0.166, 0.157, 0.086, 0.048, 0.007]
muestra.sort()
alpha = 0.05  # Nivel de confianza

# Generar las clases
clases = generar_clases(num_clases)

# genera los intervalos de clase
for i, (a, b) in enumerate(clases):
    (f"[{a:.4f}, {b:.4f}]")

#Calcula la frecuencia esperada en cada clase
def fe(N):
    fe=[]
    frec=N/len(clases)
    for  i in range(len(clases)):
        fe.append(frec)
    return fe

#calcula la frecuencia observada
def fo(clases, muestra):
    fo = [0] * len(clases)
    for valor in muestra:
        for i, (limite_inferior, limite_superior) in enumerate(clases):
            if limite_inferior <= valor < limite_superior:  # Verifica si el valor está en el rango
                fo[i] += 1  # Incrementa la frecuencia de la clase correspondiente
                break  # Pasa a la siguiente muestra
    return fo  # Devuelve el vector con las frecuencias observadas

# llama a las frecuencias observadas
frecuencias_observadas = fo(clases, muestra)

#calcula la resta de las frecuencias
def resta(fe,fo):
    res=[]
    for i in range(len(fe)):
        res.append(fo[i]-fe[i])
    return res
# llama a la resta de las frecuencias
res=resta(fe(N),frecuencias_observadas)

#Calcula el cuadrado de la resta
def res_cuad(res):
    res_cuad=[]
    for i in range(len(res)):
        res_cuad.append(res[i]**2)
    return res_cuad
# llama al cuadrado de la resta
res_cuad=res_cuad(res)

#Calcula el cuadrado de la resta y lo divide entre fe
def vec_chi(res_cuad,fe):
    vchi=[]
    for i in range(len(res_cuad)):
        opchi=res_cuad[i]/fe[i]
        vchi.append(opchi)
    return vchi
#Calcula el valor de chi^2
def chi(res_cuad,fe):
    chi=0
    for i in range(len(res_cuad)):
        chi+=res_cuad[i]/fe[i]
    return chi

print("\033[1mgrados de libertad:\033[4m", num_clases-1) #imprime los grados de libertad
#llama el valor de chi
chi_cuadrada= chi(res_cuad,fe(N))
#da los grados de libertad
grados_libertad = num_clases-1

vcritico_chi = scipy.stats.chi2.ppf(1 - alpha, grados_libertad)#da el valor critico de chi

#condición para ver si se rechaza o no la H0
if chi_cuadrada > vcritico_chi:
    print("\033[1mRechaza H0\033[0m")
else:
    print("\033[1mNo se rechaza H0\033[0m")

# Crea la tabla con los datos correspondientes
datos = [
    ["\033[1mClase\033[0m", "\033[1mfe\033[0m", "\033[1mfo\033[0m","\033[1m(fo-fe)\033[0m","\033[1m(fo-fe)^2\033[0m" , "\033[1m(fo-fe)^2/fe\033[0m"]  # Header row
]

for i, (a, b) in enumerate(clases):
    clase = f"[{a:.4f}, {b:.4f}]"
    fe_value = fe(N)[i]
    fo_value = frecuencias_observadas[i]
    rest= resta(fe(N),frecuencias_observadas)[i]
    rest_cuad=res_cuad[i]
    vec_chi_value = vec_chi(res_cuad, fe(N))[i]

    datos.append([clase, fe_value, fo_value, rest, rest_cuad ,vec_chi_value])

# Calcula los totales
total_fe = sum(fe(N))
total_fo = sum(frecuencias_observadas)

# agrega una última fila
datos.append(["\033[1mTotal\033[0m", total_fe, total_fo, "", "\033[1mchi^2=\033[0m", chi_cuadrada])

# imprime la tabla
print(tabulate.tabulate(datos, headers="firstrow", tablefmt="fancy_grid",
    numalign="center", stralign="center"))

grados de libertad: 4
No se rechaza H0
╒══════════════════╤══════╤══════╤═══════════╤═════════════╤════════════════╕
│      Clase       │  fe  │  fo  │  (fo-fe)  │  (fo-fe)^2  │  (fo-fe)^2/fe  │
╞══════════════════╪══════╪══════╪═══════════╪═════════════╪════════════════╡
│ [0.0000, 0.1999] │  5   │  7   │    2.0    │     4.0     │      0.8       │
├──────────────────┼──────┼──────┼───────────┼─────────────┼────────────────┤
│ [0.2000, 0.3999] │  5   │  3   │   -2.0    │     4.0     │      0.8       │
├──────────────────┼──────┼──────┼───────────┼─────────────┼────────────────┤
│ [0.4000, 0.5999] │  5   │  9   │    4.0    │    16.0     │      3.2       │
├──────────────────┼──────┼──────┼───────────┼─────────────┼────────────────┤
│ [0.6000, 0.7999] │  5   │  3   │   -2.0    │     4.0     │      0.8       │
├──────────────────┼──────┼──────┼───────────┼─────────────┼────────────────┤
│ [0.8000, 1.0000] │  5   │  3   │   -2.0    │     4.0     │      0.8       │
├──────────────────┼─────